In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory('OCT2017_\\train',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory('OCT2017_\\val',
                                                target_size=(224, 224),
                                                batch_size=32,
                                                class_mode='categorical')

test_generator = test_datagen.flow_from_directory('OCT2017_\\test',
                                                  target_size=(224, 224),
                                                  batch_size=1,
                                                  class_mode='categorical',
                                                  shuffle=False)


Found 83484 images belonging to 4 classes.
Found 32 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [3]:
base_model = ResNet50(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
model.fit(train_generator, epochs=10, validation_data=val_generator)


Epoch 1/10
2609/2609 [==============================] - 4290s 2s/step - loss: 0.4809 - accuracy: 0.8278 - val_loss: 0.5934 - val_accuracy: 0.7188
Epoch 2/10
2609/2609 [==============================] - 4276s 2s/step - loss: 0.3902 - accuracy: 0.8570 - val_loss: 0.5739 - val_accuracy: 0.6875
Epoch 3/10
2609/2609 [==============================] - 4278s 2s/step - loss: 0.3686 - accuracy: 0.8658 - val_loss: 0.3108 - val_accuracy: 0.8125
Epoch 4/10
2609/2609 [==============================] - 4275s 2s/step - loss: 0.3504 - accuracy: 0.8719 - val_loss: 0.2633 - val_accuracy: 0.9062
Epoch 5/10
2609/2609 [==============================] - 4281s 2s/step - loss: 0.3428 - accuracy: 0.8759 - val_loss: 0.2891 - val_accuracy: 0.9062
Epoch 6/10
2609/2609 [==============================] - 4283s 2s/step - loss: 0.3353 - accuracy: 0.8786 - val_loss: 0.2966 - val_accuracy: 0.9062
Epoch 7/10
2609/2609 [==============================] - 4294s 2s/step - loss: 0.3293 - accuracy: 0.8820 - val_loss: 0.3702 -

In [5]:
model.evaluate(test_generator)


968/968 [==============================] - 87s 89ms/step - loss: 0.2499 - accuracy: 0.8853


[0.24992519617080688, 0.8853305578231812]